# Setup

In [3816]:
import os
import sys
from pathlib import Path
import arff
import pandas as pd
import numpy as np

In [3817]:
sys.path.append(os.path.abspath("../../../.."))

In [3818]:
from src.experiment.helpers.variables import dataset_root_dir
from src.experiment.helpers.utils import get_sorted_class_percentages, get_cardinality

In [3819]:
NAME = "emotions"

In [3820]:
DATASET_DIR = dataset_root_dir / 'multilabel' / NAME
print(DATASET_DIR)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multilabel\emotions


In [3821]:
balanced_name = NAME + "_balanced"
DATASET_DIR_BALANCED = dataset_root_dir / 'multilabel' / balanced_name
print(DATASET_DIR_BALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multilabel\emotions_balanced


In [3822]:
unbalanced_name = NAME + "_unbalanced"
DATASET_DIR_UNBALANCED = dataset_root_dir / 'multilabel' / unbalanced_name
print(DATASET_DIR_UNBALANCED)

c:\VisualStudioRepositories\MUSIC_DATA\datasets\multilabel\emotions_unbalanced


# Dataset loading

In [3931]:
X = pd.read_csv(DATASET_DIR / 'X.csv')
y = pd.read_csv(DATASET_DIR / 'y.csv')

In [3932]:
X

,Mean_Acc1298_Mean_Mem40_Centroid,Mean_Acc1298_Mean_Mem40_Rolloff,Mean_Acc1298_Mean_Mem40_Flux,Mean_Acc1298_Mean_Mem40_MFCC_0,Mean_Acc1298_Mean_Mem40_MFCC_1,Mean_Acc1298_Mean_Mem40_MFCC_2,Mean_Acc1298_Mean_Mem40_MFCC_3,Mean_Acc1298_Mean_Mem40_MFCC_4,Mean_Acc1298_Mean_Mem40_MFCC_5,Mean_Acc1298_Mean_Mem40_MFCC_6,...,Std_Acc1298_Std_Mem40_MFCC_11,Std_Acc1298_Std_Mem40_MFCC_12,BH_LowPeakAmp,BH_LowPeakBPM,BH_HighPeakAmp,BH_HighPeakBPM,BH_HighLowRatio,BHSUM1,BHSUM2,BHSUM3
0,0.034741,0.089665,0.091225,-73.302422,6.215179,0.615074,2.037160,0.804065,1.301409,0.558576,...,0.118630,0.094923,0.051035,68.0,0.014937,136.0,2.0,0.245457,0.105065,0.405399
1,0.081374,0.272747,0.085733,-62.584437,3.183163,-0.218145,0.163038,0.620251,0.458514,0.041426,...,0.070075,0.041565,0.295031,70.0,0.276366,140.0,2.0,0.343547,0.276366,0.710924
2,0.110545,0.273567,0.084410,-65.235325,2.794964,0.639047,1.281297,0.757896,0.489412,0.627636,...,0.079917,0.085821,0.161574,61.0,0.000000,183.0,3.0,0.188693,0.045941,0.457372
3,0.042481,0.199281,0.093447,-80.305152,5.824409,0.648848,1.754870,1.495532,0.739909,0.809644,...,0.129145,0.122330,0.043012,66.0,0.206562,132.0,2.0,0.102839,0.241934,0.351009
4,0.074550,0.140880,0.079789,-93.697749,5.543229,1.064262,0.899152,0.890336,0.702328,0.490685,...,0.284196,0.189988,0.029308,100.0,0.144039,200.0,2.0,0.195196,0.310801,0.683817
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,0.027142,0.047551,0.072043,-79.881347,8.119313,1.927310,1.696017,0.397888,0.857559,0.302742,...,0.129552,0.106260,0.089991,87.0,0.564842,174.0,2.0,0.261742,0.002657,1.149211
589,0.094829,0.204498,0.082824,-61.364436,2.966229,0.627740,1.440352,0.856243,1.110282,0.394450,...,0.128090,0.167160,0.035827,65.0,0.003942,130.0,2.0,0.282122,0.052218,0.335371
590,0.035169,0.065403,0.075227,-81.750533,10.311701,0.092224,0.818851,1.569606,1.831909,0.057216,...,0.147194,0.098238,0.033681,88.0,0.000000,176.0,2.0,0.184313,0.247136,0.476993
591,0.054276,0.238158,0.095935,-71.009724,3.181340,1.547197,2.407780,0.618838,0.997950,0.825143,...,0.080006,0.058181,0.155650,84.0,0.276695,168.0,2.0,0.547126,0.183494,1.255820


In [3933]:
y

,relaxing-calm,angry-aggresive,amazed-suprised,sad-lonely,happy-pleased,quiet-still
0,1,0,0,0,1,0
1,0,1,1,0,0,0
2,0,1,0,0,1,0
3,1,0,0,0,0,0
4,0,0,0,0,0,1
...,...,...,...,...,...,...
588,1,0,0,1,0,1
589,0,1,1,1,0,0
590,1,0,0,1,0,1
591,1,0,0,0,1,0


# Dataset info

In [3826]:
# number of samples
len(X)

593

In [3827]:
# number of classes
len(y.columns)

6

In [3828]:
class_percentages_sorted = get_sorted_class_percentages(y)
print(class_percentages_sorted)

relaxing-calm      44.519393
angry-aggresive    31.871838
amazed-suprised    29.173693
sad-lonely         28.330523
happy-pleased      27.993255
quiet-still        24.957841
dtype: float64


In [3829]:
cardinality = get_cardinality(y)
print("Cardinality:", cardinality)

Cardinality: 1.8684654300168635


# Functions

In [3830]:
def get_least_represented_class_names(class_percentages_sorted, percent):
    num_classes = len(class_percentages_sorted)
    bottom_percent_count = int(np.ceil(num_classes * percent))
    least_represented_classes = class_percentages_sorted.tail(bottom_percent_count).index.tolist()
    return least_represented_classes

In [3831]:
# removes SAMPLES
def remove_few_samples_of_class(X, y, class_name, step, protected_classes):
    y_numeric = y.apply(pd.to_numeric)

    available_indices = y_numeric.index[
        (y_numeric[class_name] == 1) &
        (y_numeric[protected_classes].sum(axis=1) == 0)
    ].tolist()

    np.random.seed(47)
    remove_indices = np.random.choice(available_indices , step, replace=False)

    X_trimmed = X.drop(remove_indices).reset_index(drop=True)
    y_trimmed = y.drop(remove_indices).reset_index(drop=True)
    
    return X_trimmed, y_trimmed

In [3832]:
# removes a CLASS
# if samples that belong only to that class exist, they will be deleted
# samples that belong to multiple classes will be kept
def remove_class(X, y, least_represented=True):
    class_counts = y.sum()
    target_class = class_counts.idxmin() if least_represented else class_counts.idxmax()
    # Find samples where only this class is present
    only_target = (y.sum(axis=1) == 1) & (y[target_class] == 1)
    # Remove those samples
    X_new = X.loc[~only_target].reset_index(drop=True)
    y_new = y.loc[~only_target].drop(columns=[target_class]).reset_index(drop=True)
    return X_new, y_new

In [3833]:
# removes CLASSES
# if samples that belong only to those classes exist, they will be deleted
# samples that still have other classes will be kept
def remove_bottom_percent_classes(X, y, percent):
    class_percentages_sorted = get_sorted_class_percentages(y)
    bottom_classes = get_least_represented_class_names(class_percentages_sorted, percent)

    # Remove the bottom classes from y
    y_new = y.drop(columns=bottom_classes)

    # Find samples that would have zero classes after dropping bottom classes
    zero_class_samples = y_new.sum(axis=1) == 0

    # Remove those samples from both X and y_new
    X_new = X.loc[~zero_class_samples].reset_index(drop=True)
    y_new = y_new.loc[~zero_class_samples].reset_index(drop=True)

    return X_new, y_new

In [3834]:
def show_summary(X, y):
    print("Number of samples:", len(X))
    print("Number of classes:", y.shape[1])
    
    cardinality = get_cardinality(y)
    print("Cardinality:", round(cardinality, 3))
    
    num_samples_no_labels = (y.sum(axis=1) == 0).sum()
    print("Samples with no labels:", num_samples_no_labels)

    class_percentages_sorted = get_sorted_class_percentages(y)
    print(class_percentages_sorted)
    return class_percentages_sorted

In [3835]:
# removes SAMPLES
def remove_samples_with_one_class(X, y, step, protected_classes, random_state=42):
    y_numeric = y.apply(pd.to_numeric)

    # Find indices where none of the protected classes are present
    available_indices = y_numeric.index[
        (y_numeric[protected_classes].sum(axis=1) == 0)
    ]

    # From those, select indices where only one class is present
    single_class_indices = y_numeric.loc[available_indices].index[
        y_numeric.loc[available_indices].sum(axis=1) == 1
    ].tolist()
    
    if step > len(single_class_indices):
        raise ValueError(f"Requested to remove {step} samples, but only {len(single_class_indices)} single-class samples from non-protected classes are available.")
    
    np.random.seed(random_state)
    remove_indices = np.random.choice(single_class_indices, step, replace=False)
    X_new = X.drop(remove_indices).reset_index(drop=True)
    y_new = y.drop(remove_indices).reset_index(drop=True)
    return X_new, y_new

In [3836]:
def remove_samples_with_zero_classes(X, y, step, random_state=42):
    y_numeric = y.apply(pd.to_numeric)
    zero_class_indices = y_numeric.index[y_numeric.sum(axis=1) == 0].tolist()
    
    if step > len(zero_class_indices):
        raise ValueError(f"Requested to remove {step} samples, but only {len(zero_class_indices)} samples with zero classes are available.")
    
    np.random.seed(random_state)
    remove_indices = np.random.choice(zero_class_indices, step, replace=False)
    X_new = X.drop(remove_indices).reset_index(drop=True)
    y_new = y.drop(remove_indices).reset_index(drop=True)
    return X_new, y_new

# Dataset trimming

In [3450]:
X, y  = remove_bottom_percent_classes(X, y, percent=0.2)
class_percentages_sorted = show_summary(X, y)

Number of samples: 14464
Number of classes: 12
Cardinality: 3.794
Samples with no labels: 0
TAG_reference      44.904591
TAG_web            41.648230
TAG_tools          40.742533
TAG_software       35.342920
TAG_design         33.600664
TAG_blog           33.427821
TAG_howto          27.087942
TAG_free           26.839049
TAG_development    25.608407
TAG_programming    24.764934
TAG_web2.0         23.264657
TAG_cool           22.137721
dtype: float64


In [3305]:
X, y  = remove_class(X, y, least_represented=True)
class_percentages_sorted = show_summary(X, y)

Number of samples: 14986
Number of classes: 23
Cardinality: 5.573
Samples with no labels: 0
TAG_reference      43.340451
TAG_web            40.197518
TAG_tools          39.323368
TAG_software       34.111838
TAG_design         32.430268
TAG_blog           32.263446
TAG_howto          26.144401
TAG_free           25.904177
TAG_development    24.716402
TAG_programming    23.902309
TAG_web2.0         22.454291
TAG_cool           21.366609
TAG_tutorial       21.046310
TAG_webdesign      19.925264
TAG_imported       19.811824
TAG_blogs          18.417189
TAG_internet       18.076872
TAG_tips           17.135994
TAG_art            15.801415
TAG_fun            15.667957
TAG_webdev         15.627919
TAG_opensource     14.953957
TAG_technology     14.713733
dtype: float64


In [2096]:
X, y  = remove_class(X, y, least_represented=False)
class_percentages_sorted = show_summary(X, y)

Number of samples: 3939
Number of classes: 40
Cardinality: 1.651
Samples with no labels: 0
mountain    8.758568
snow        7.565372
flowers     7.514598
clouds      7.108403
rocks       6.346789
stone       5.915207
street      5.813658
plane       5.686723
bear        5.585174
field       5.458238
sand        5.153592
birds       4.975882
beach       4.950495
boats       4.315816
jet         4.214268
cars        3.833460
leaf        3.757299
plants      3.655750
house       3.630363
bridge      3.503427
polar       3.427266
valley      3.376491
hills       3.325717
garden      3.224169
close-up    3.097233
ruins       3.021071
statue      2.970297
horses      2.919523
tracks      2.894136
wall        2.817974
sun         2.817974
ice         2.817974
cat         2.716425
temple      2.665651
ocean       2.665651
train       2.665651
tiger       2.564103
scotland    2.538715
coral       2.487941
swimmers    2.361005
dtype: float64


In [4085]:
class_percentages_sorted = get_sorted_class_percentages(y)

least_represented_classes = get_least_represented_class_names(class_percentages_sorted, 0.3)
most_represented_class = class_percentages_sorted.index[0]
print("Most populated class:", most_represented_class)

X, y = remove_few_samples_of_class(X, y, most_represented_class, 1, least_represented_classes)
class_percentages_sorted = show_summary(X, y)

Most populated class: angry-aggresive
Number of samples: 450
Number of classes: 6
Cardinality: 1.933
Samples with no labels: 0
relaxing-calm      33.777778
angry-aggresive    33.777778
amazed-suprised    33.777778
sad-lonely         30.666667
happy-pleased      30.666667
quiet-still        30.666667
dtype: float64


In [3902]:
least_represented_classes = get_least_represented_class_names(class_percentages_sorted, 0.3)

X, y = remove_samples_with_one_class(X, y, step=2, protected_classes=least_represented_classes)
class_percentages_sorted = show_summary(X, y)

Number of samples: 471
Number of classes: 6
Cardinality: 2.093
Samples with no labels: 0
relaxing-calm      47.133758
sad-lonely         33.121019
happy-pleased      32.696391
angry-aggresive    32.484076
amazed-suprised    32.484076
quiet-still        31.422505
dtype: float64


In [3813]:
X, y = remove_samples_with_zero_classes(X, y, step=100000)
class_percentages_sorted = show_summary(X, y)

ValueError: Requested to remove 100000 samples, but only 0 samples with zero classes are available.

# Trimmed dataset - info

In [4086]:
class_percentages_sorted = show_summary(X, y)

Number of samples: 450
Number of classes: 6
Cardinality: 1.933
Samples with no labels: 0
relaxing-calm      33.777778
angry-aggresive    33.777778
amazed-suprised    33.777778
sad-lonely         30.666667
happy-pleased      30.666667
quiet-still        30.666667
dtype: float64


# Save

### Balanced

In [4087]:
DATASET_DIR_BALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_BALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_BALANCED / "y.csv", index=False)

In [4088]:
class_percentages_sorted.to_csv(DATASET_DIR_BALANCED / "class_percentages.csv", header=False)

### Unbalanced

In [ ]:
DATASET_DIR_UNBALANCED.mkdir(parents=True, exist_ok=True)
X.to_csv(DATASET_DIR_UNBALANCED / "X.csv", index=False)
y.to_csv(DATASET_DIR_UNBALANCED / "y.csv", index=False)

In [ ]:
class_percentages_sorted.to_csv(DATASET_DIR_UNBALANCED / "class_percentages.csv", header=False)